In [2]:
import pandas as pd
from datetime import datetime

In [3]:
auxilios = pd.read_excel('dados/auxilios_novembro_2025.xlsx', skiprows=1)

In [4]:
auxilios.head()

,APS,APS.1,Competência concessão,Espécie,Espécie.1,CID,CID.1,Despacho,Despacho.1,Dt Nascimento,...,Ramo Atividade,Dt DCB,Dt DDB,Dt DIB,País de Acordo Internacional,Classificador PA,CNAE 2.0,CNAE 2.0.1,Grau Instrução,Qt Anos Contribuição
0,2001040,02001040-Aps Maceio - Jatiuca,202511,31,Auxílio Doenca Previdenciário,D251,D25.1 Leiomioma Intramural do Utero,64,Concessao com Analise Documental,1984-07-04,...,Comerciario,2025-10-25 00:00:00,2025-11-01,2025-10-11,{ñ class},Sem Pensâo Alimentícia,99990,{ñ class},Não Informado,9
1,2001030,02001030-Aps Maceio - Almirante Alvaro Calhe,202511,91,Auxílio Doenca por Acidente do Trabalho,F331,F33.1 Transt Depress Recorr Episodio Atual Mo,64,Concessao com Analise Documental,1990-09-01,...,Comerciario,2025-12-29 00:00:00,2025-11-01,2025-10-31,{ñ class},Sem Pensâo Alimentícia,99990,{ñ class},Não Informado,11
2,2001110,02001110-Aps Uniao dos Palmares,202511,31,Auxílio Doenca Previdenciário,M23,M23 Transt Internos dos Joelhos,64,Concessao com Analise Documental,1987-12-29,...,Comerciario,2025-12-13 00:00:00,2025-11-01,2025-10-30,{ñ class},Sem Pensâo Alimentícia,99990,{ñ class},Não Informado,0
3,2001070,02001070-Aps Penedo,202511,31,Auxílio Doenca Previdenciário,S92,S92 Frat do Pe,64,Concessao com Analise Documental,1987-05-05,...,Comerciario,2025-12-19 00:00:00,2025-11-01,2025-10-22,{ñ class},Sem Pensâo Alimentícia,99990,{ñ class},Não Informado,11
4,2001040,02001040-Aps Maceio - Jatiuca,202511,31,Auxílio Doenca Previdenciário,M511,M51.1 Transt Disco Lombar Outr Intervert Radi,64,Concessao com Analise Documental,1991-03-08,...,Comerciario,2025-12-21 00:00:00,2025-11-01,2025-10-23,{ñ class},Sem Pensâo Alimentícia,99990,{ñ class},Não Informado,2


In [5]:
auxilios.columns

Index(['APS', 'APS.1', 'Competência concessão', 'Espécie', 'Espécie.1', 'CID',
       'CID.1', 'Despacho', 'Despacho.1', 'Dt Nascimento', 'Sexo.',
       'Clientela', 'Mun Resid', 'Vínculo dependentes', 'Forma Filiação', 'UF',
       'Qt SM RMI', 'Ramo Atividade', 'Dt DCB', 'Dt DDB', 'Dt DIB',
       'País de Acordo Internacional', 'Classificador PA', 'CNAE 2.0',
       'CNAE 2.0.1', 'Grau Instrução', 'Qt Anos Contribuição'],
      dtype='object')

In [6]:
def limpeza_auxilios(auxilios): 
    auxilios = auxilios.copy()

    auxilios.columns = (
        auxilios.columns
        .str.strip() #remove os espaços antes e depois
        .str.normalize('NFKD') #remove acentos
    )
    
    colunas_para_remover = [
        'APS', 'APS.1', 'Competência concessão', 'Espécie', 'CID', 'Despacho', 'Mun Resid', 'País de Acordo Internacional', 'CNAE 2.0', 'CNAE 2.0.1'
    ]
    auxilios.drop(columns=colunas_para_remover, inplace=True, errors='ignore')

    renomear_colunas = {
        'Espécie.1' : 'Tipos de auxilio', 
        'CID.1' : 'CID',
        'Despacho.1' : 'Despacho',
        'Dt Nascimento' : 'Data nasc',
        'Sexo.' : 'Sexo',
        'Qt SM RMI' : 'Quant salarios minimos na RMI',
        'Dt DCB' : 'Data DCB',
        'Dt DDB' : 'Data DDB',
        'Dt DIB' : 'Data DIB',
        'Classificador PA' : 'Pensão Alimenticia'
    }
    auxilios.rename(columns=renomear_colunas, inplace=True)

    #arredonda para duas casas decimais
    auxilios['Quant salarios minimos na RMI'] = auxilios['Quant salarios minimos na RMI'].round(2)

    #transformar o que é string para datetime
    for col in ["Data DCB", "Data DDB", "Data DIB", "Data nasc"]:
        auxilios[col] = pd.to_datetime(auxilios[col], errors="coerce", dayfirst=True)

    #onde tem valores faltando colocar 'nao informado'
    auxilios['CID'] = auxilios['CID'].fillna('Não informado')

    #células vazias de data final sejam substituidas pela data atual
    auxilios['Data DCB'] = auxilios['Data DCB'].fillna(datetime.now())
    auxilios['Duração'] = auxilios['Data DCB'] - auxilios['Data DIB']
    
    #apaga a duplicada e deixa a original
    auxilios.drop_duplicates(inplace=True)

    # Reordenar DIB(inicio), DDB(despacho/analise), DCB(fim)
    auxilios = auxilios[
        [c for c in auxilios.columns if c not in ["Data DCB", "Data DDB", "Data DIB"]]
        + ["Data DIB", "Data DDB", "Data DCB"]
    ]

    return auxilios


In [7]:
auxilios = limpeza_auxilios(auxilios)

In [8]:
auxilios

,Tipos de auxilio,CID,Despacho,Data nasc,Sexo,Clientela,Vínculo dependentes,Forma Filiação,UF,Quant salarios minimos na RMI,Ramo Atividade,Pensão Alimenticia,Grau Instrução,Qt Anos Contribuição,Duração,Data DIB,Data DDB,Data DCB
0,Auxílio Doenca Previdenciário,D25.1 Leiomioma Intramural do Utero,Concessao com Analise Documental,1984-07-04,Feminino,Urbano,Não Informado,Empregado,Alagoas,1.50,Comerciario,Sem Pensâo Alimentícia,Não Informado,9,14 days 00:00:00,2025-10-11,2025-11-01,2025-10-25 00:00:00.000000
1,Auxílio Doenca por Acidente do Trabalho,F33.1 Transt Depress Recorr Episodio Atual Mo,Concessao com Analise Documental,1990-09-01,Feminino,Urbano,Não Informado,Empregado,Alagoas,2.06,Comerciario,Sem Pensâo Alimentícia,Não Informado,11,59 days 00:00:00,2025-10-31,2025-11-01,2025-12-29 00:00:00.000000
2,Auxílio Doenca Previdenciário,M23 Transt Internos dos Joelhos,Concessao com Analise Documental,1987-12-29,Feminino,Urbano,Não Informado,Empregado,Alagoas,1.00,Comerciario,Sem Pensâo Alimentícia,Não Informado,0,44 days 00:00:00,2025-10-30,2025-11-01,2025-12-13 00:00:00.000000
3,Auxílio Doenca Previdenciário,S92 Frat do Pe,Concessao com Analise Documental,1987-05-05,Feminino,Urbano,Não Informado,Empregado,Alagoas,1.46,Comerciario,Sem Pensâo Alimentícia,Não Informado,11,58 days 00:00:00,2025-10-22,2025-11-01,2025-12-19 00:00:00.000000
4,Auxílio Doenca Previdenciário,M51.1 Transt Disco Lombar Outr Intervert Radi,Concessao com Analise Documental,1991-03-08,Masculino,Urbano,Não Informado,Desempregado,Alagoas,1.00,Comerciario,Sem Pensâo Alimentícia,Não Informado,2,59 days 00:00:00,2025-10-23,2025-11-01,2025-12-21 00:00:00.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584916,Auxílio Doenca Previdenciário,F41.2 Transt Misto Ansioso e Depressivo,Concessao com Analise Documental,1997-10-04,Feminino,Urbano,Não Informado,Desempregado,Tocantins,1.08,Comerciario,Sem Pensâo Alimentícia,Não Informado,2,29 days 00:00:00,2025-11-24,2025-11-29,2025-12-23 00:00:00.000000
584917,Auxílio Doenca Previdenciário,N80 Endometriose,Concessao com Analise Documental,1988-07-24,Feminino,Urbano,Não Informado,Empregado,Tocantins,1.00,Comerciario,Sem Pensâo Alimentícia,Não Informado,6,44 days 00:00:00,2025-11-19,2025-11-29,2026-01-02 00:00:00.000000
584918,Auxílio Doenca Previdenciário,G56.0 Sindr do Tunel do Carpo,Concessao com Analise Documental,1973-11-23,Feminino,Urbano,Não Informado,Autônomo,Tocantins,1.00,Comerciario,Sem Pensâo Alimentícia,Não Informado,11,59 days 00:00:00,2025-10-27,2025-11-29,2025-12-25 00:00:00.000000
584919,Amp. Social Pessoa Portadora Deficiencia,Zerados,Concessao Decorrente de Acao Judicial,1978-02-13,Masculino,Rural,Não Informado,Desempregado,Tocantins,1.00,Rural,Sem Pensâo Alimentícia,Não Informado,0,556 days 13:30:42.874522,2024-08-13,2025-11-29,2026-02-20 13:30:42.874522


In [9]:
auxilios.to_csv('dados/dados_limpos.csv')

In [10]:
auxilios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 580803 entries, 0 to 584920
Data columns (total 18 columns):
 #   Column                         Non-Null Count   Dtype          
---  ------                         --------------   -----          
 0   Tipos de auxilio               580803 non-null  object         
 1   CID                            580803 non-null  object         
 2   Despacho                       580803 non-null  object         
 3   Data nasc                      580803 non-null  datetime64[ns] 
 4   Sexo                           580803 non-null  object         
 5   Clientela                      580803 non-null  object         
 6   Vínculo dependentes           580803 non-null  object         
 7   Forma Filiação               580803 non-null  object         
 8   UF                             580803 non-null  object         
 9   Quant salarios minimos na RMI  580803 non-null  float64        
 10  Ramo Atividade                 580803 non-null  object       

In [11]:
#retorna True se qualquer coluna tiver nulo (Nan ou None)
#auxilios.isna().any().any()

In [12]:
#retorna quantos nulos tem por coluna
#auxilios.isna().sum()

In [13]:
#mostra quantidade de duplicada
#auxilios.duplicated().sum()

In [14]:
#mostra as colunas duplicadas, a original e a outra 
#auxilios[auxilios.duplicated(keep=False)]

In [15]:
#retorna somente as colunas duplicadas, sem mostrar a original
#auxilios[auxilios.duplicated()]

In [17]:
#lê o arquivo de dados populacional e faz um pequeno tratamento 
populacao = pd.read_excel('dados/dados_populacional.xlsx',skiprows=3)
populacao['População_residente'] = populacao['População_residente'].astype(int)
populacao["Unidade da Federação"] = populacao["Unidade da Federação"].str.split(" ", n=1).str[1] #split divide no primeiro espaço e str pegao 2 elemento (indice 1)

#Remove as linhas com NaN em qualquer coluna
populacao = populacao.dropna()

populacao = populacao.sort_values('População_residente', ascending=False)
populacao.to_csv('dados/dados_populacional_limpo.csv')